In [78]:
import yaml
import os
import glob
import json
from datetime import datetime

prj = "Z:/Amite/Amite_LWI/Models/Amite_HMS/Amite_HMS.hms"
prj_dir, prj_file_tail = os.path.split(prj)
prj_name = prj_file_tail.split(".")[0]

with open("Z:/Amite/Amite_LWI/Models/Amite_HMS/Amite_HMS.hms", "r") as f:
     lines = f.readlines()
lines = [s.strip('\n') for s in lines]
# lines

In [79]:
import copy
keyValueList = copy.deepcopy(lines)  

nest_start = 0
nestList = []
for i,v in enumerate(lines):
        if v == 'End:':
                # If not the beginning of the file, skip a blank line (+1) for the start of the subList.
                if len(nestList) > 0:
                        nestList.append(lines[nest_start+1:i])
                else:
                        nestList.append(lines[nest_start:i])
                nest_start = i+1
# nestList



In [34]:
# Get the unique headers [Project, Control, Basin, etc.] as a list.
headers = []
for subList in nestList:
    headers.append(subList[0].split(":")[0])
unique_headers = list(set(headers))
unique_headers


['Control', 'Project', 'Basin', 'Precipitation']

In [57]:
# Create a dictionary based on keys using the unique headers with values of Title, Filename, and Description
kv = {}
kv['Control Files'] = {}
kv['Basin'] = {}
kv['Precipitaion'] = {}
headers_with_same_parsing = ['Control', 'Basin', 'Precipitation']

for subList in nestList:
    header = subList[0].split(":")[0]
    title = subList[0].split(":")[1]
    find_str = 'Description'
    description = [s for s in subList if find_str in s][0].split(":")[1:][0].strip()

    if header == 'Project':
        find_str = 'File Name'
        filename = [s for s in subList if find_str in s][0].split(":")[1:][0].strip()
        kv['Project'] = {
            'Title': title,
            'Project Output DSS File': filename,
            'Description': description
        }
    
    if any(header == i for i in headers_with_same_parsing):
        find_str = 'filename'
        filename = [s for s in subList if find_str in s.lower()][0].split(":")[1:][0].strip()
        kv['Project'][title] = {
            'File Name': filename,
            'Description': description
        }

kv.keys()

dict_keys(['Control Files', 'Basin', 'Precipitaion', 'Project'])

In [65]:
import glob, os
# prj_name = kv['Project']['Title']
# Get project files
prj_files_List = []

for pFile in glob.glob(rf'{prj_dir}/{prj_name}.*'):
        prj_files_List.append(pFile)

prj_files_List

['Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.access',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.dss',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.gage',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.hms',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.log',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.out',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.pdata',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.run']

In [81]:
# Add application_date from prj_file modified date
modTimeUnix = os.path.getmtime(prj) 
kv['application_date'] = datetime.fromtimestamp(modTimeUnix).strftime('%Y-%m-%d')

# if args.dss, get dss input files
input_dss_dir = r"Z:\Amite\Amite_LWI\Models\Amite_HMS\data"
dss_files_List = []
for pFile in glob.glob(rf'{input_dss_dir}/*.dss'):
        dss_files_List.append(pFile)

dss_files_List

['Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\Amite_HMS_TimeSeries.dss',
 'Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\rainfall_stageIV.dss',
 'Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\Specified_Hyetographs.dss',
 'Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\USGS_observed.dss']

In [75]:
# open the Json template, del unnecessary keys, update, add, export 

with open(r"C:\py\hec_meta_extract\example\input\json\hms_model_application.json", 'r') as f:
            model_template_json = json.load(f)
# model_template_json
# keys to drop from json template
drop_keys = ['_id', 'linked_resources', 'common_parameters', 'common_software_version', 'authors', 
'spatial_extent_resolved', 'spatial_valid_extent_resolved', 'temporal_extent', 'temporal_resolution', 'spatial_valid_extent', 'common_files_details']
for key in drop_keys:
    del model_template_json[key]



# set basic keywords
model_template_json['keywords'] = ['hec-hms','hec','hms','hydrology','model','lwi']

# common_files_details[]
model_template_json['common_files_details'] = []
model_template_json['common_files_details'].append(
    {
        "description": "The HMS Project File",
        "location": prj_file_tail,
        "source_dataset": None,
        "title": "Project File"
    },
    {
        "description": "There may be multiple basins in the HMS model project",
        "location": f"{prj_dir}/*.basin",
        "source_dataset": None,
        "title": "Basin Files"
    },
    {
        "description": "There may be multiple Meteorological Models",
        "location": f"{prj_dir}/*.met",
        "source_dataset": None,
        "title": "Meteorological Model Files"
    },
    {
        "description": "There may be multiple control specifications.",
        "location": f"{prj_dir}/*.control",
        "source_dataset": None,
        "title": "Control Specification Files"
    },
)


# Map kv dictionary to the model application json
# application_date, spatial_extent[0], common_files_details[..], purpose, description, title